In [19]:
#Imports
import os
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import drive

In [2]:
#Mount and access drive
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive/Few-Shot-Tamil')
print("Change successful.")

Mounted at /content/drive
Change successful.


In [3]:
def tokenize(data):
  generated_token = list(data.split())
  return generated_token

In [4]:
def tamil_preprocessing(dataframe):
  #Remove punctuation symbols
  dataframe['non_punc'] = dataframe['News'].apply(lambda x: re.sub(r'[\'\"\.?!‘]+', '', x.lower()))

  #Tokenize the sentences
  dataframe['tamil_tokenized_text'] = dataframe['non_punc'].apply(lambda x: tokenize(x))

In [5]:
#Apply TF-IDF
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()
    train = tfidf_vectorizer.fit_transform(data)
    return train, tfidf_vectorizer

In [6]:
#Load the datasets
one_shot_real = pd.read_csv('one-shot-real.csv',  sep='\t')
one_shot_fake = pd.read_csv('one-shot-fake.csv',  sep='\t')
three_shot_real = pd.read_csv('three-shot-real.csv',  sep='\t')
three_shot_fake = pd.read_csv('three-shot-fake.csv',  sep='\t')
five_shot_real = pd.read_csv('five-shot-real.csv',  sep='\t')
five_shot_fake = pd.read_csv('five-shot-fake.csv',  sep='\t')

In [7]:
#Combine the datasets
one_shot = pd.concat([one_shot_real, one_shot_fake])
three_shot = pd.concat([three_shot_real, three_shot_fake])
five_shot = pd.concat([five_shot_real, five_shot_fake])

In [8]:
one_shot['Authenticity'] = [0, 0, 0, 0, 0, 1, 1, 1 ,1, 1]
three_shot['Authenticity'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1]
five_shot['Authenticity'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1, 1, 1, 1 ,1, 1]

In [9]:
#Change the label names
one_shot['Label'] = one_shot['Label'].apply(lambda x: re.sub('[^a-zA-Z]+', '', str(x)))
one_shot['Label'].unique()

array(['entertainment', 'miscellaneous', 'politics', 'sport', 'tech'],
      dtype=object)

In [10]:
#Change the label names
three_shot['Label'] = three_shot['Label'].apply(lambda x: re.sub('[^a-zA-Z]+', '', str(x)))
three_shot['Label'].unique()

array(['entertainment', 'miscellaneous', 'politics', 'sport', 'tech'],
      dtype=object)

In [11]:
#Change the label names
five_shot['Label'] = five_shot['Label'].apply(lambda x: re.sub('[^a-zA-Z]+', '', str(x)))
five_shot['Label'].unique()

array(['entertainment', 'miscellaneous', 'politics', 'sport', 'tech'],
      dtype=object)

In [12]:
#Take 10% of the original dataset for testing
os.chdir('/content/drive/My Drive')
news_df = pd.read_csv(filepath_or_buffer='news.csv')
X = news_df['News']
y = news_df['Authenticity']
xtrain, xtest, ytrain, ytest = train_test_split(X, y, stratify=y, random_state=42, test_size=0.1,)

In [13]:
ytest

2451    1
870     0
455     0
5158    1
1257    0
       ..
3884    1
2411    1
775     0
4607    1
1863    0
Name: Authenticity, Length: 523, dtype: int64

In [14]:
ytest.shape

(523,)

In [15]:
#Apply TF-IDF to the dataset
tamil_preprocessing(one_shot)
one_shot.head()

,English version,Label,News,Author,Date,URL,Authenticity,non_punc,tamil_tokenized_text
0,Actor Jayam Ravi has Corona,entertainment,நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,0,நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]"
1,A football god about the 'hands of God',miscellaneous,‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...,0,கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,"[கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]"
2,Cameras mandatory in school buses: Tamil Nadu ...,politics,பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,செய்திப்பிரிவு,21-Oct-22,https://www.hindutamil.in/news/tamilnadu/88576...,0,பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,"[பள்ளி, பேருந்துகளில், கேமரா, கட்டாயம்:, தமிழக..."
3,36th National Games | 4 gold medals for Tamil ...,sport,36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,செய்திப்பிரிவு,02-Oct-22,https://www.hindutamil.in/news/sports/877427-3...,0,36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,"[36-வது, தேசிய, விளையாட்டு, |, ஒரே, நாளில், தம..."
4,Introducing NoiseFit Core 2 Smartwatch: Amazin...,tech,நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,செய்திப்பிரிவு,22-Aug-22,https://www.hindutamil.in/news/technology/8467...,0,நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,"[நாய்ஸ்ஃபிட், கோர், 2, ஸ்மார்ட்வாட்ச், அறிமுகம..."


### One-Shot dataset

In [23]:
one_shot['clean'] = one_shot['tamil_tokenized_text'].apply(lambda x: ' '.join(x))

In [24]:
one_shot_x, tfidf_vectorizer = tfidf(one_shot['clean'])
one_shot_test = tfidf_vectorizer.transform(xtest)

In [26]:
#Run the machine learning models for one-shot
#Logistic Regression with TF-IDF
clf = LogisticRegression(C=1.0)
clf.fit(one_shot_x, one_shot['Authenticity'])
predictions = clf.predict_proba(one_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.6328871892925431

In [27]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.79      0.56      0.66       329
           1       0.50      0.75      0.60       194

    accuracy                           0.63       523
   macro avg       0.65      0.66      0.63       523
weighted avg       0.69      0.63      0.64       523



In [30]:
#Naive Bayes
clf = MultinomialNB(alpha=1.0)
clf.fit(one_shot_x, one_shot['Authenticity'])
predictions = clf.predict_proba(one_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.6309751434034416

In [31]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.76      0.56      0.65       316
           1       0.52      0.73      0.61       207

    accuracy                           0.63       523
   macro avg       0.64      0.65      0.63       523
weighted avg       0.67      0.63      0.63       523



In [32]:
#Support Vector Machine
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(one_shot_x, one_shot['Authenticity'])
predictions = clf.predict_proba(one_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.367112810707457

In [33]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.22      0.26      0.24       202
           1       0.48      0.44      0.46       321

    accuracy                           0.37       523
   macro avg       0.35      0.35      0.35       523
weighted avg       0.38      0.37      0.37       523



In [34]:
#XG-Boost classifier
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(one_shot_x, one_shot['Authenticity'])
predictions = clf.predict_proba(one_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy


0.44550669216061184

In [35]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       1.00      0.45      0.62       523
           1       0.00      0.00      0.00         0

    accuracy                           0.45       523
   macro avg       0.50      0.22      0.31       523
weighted avg       1.00      0.45      0.62       523



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Three-Shot dataset

In [37]:
#Apply TF-IDF to the dataset
tamil_preprocessing(three_shot)
three_shot.head()

,English version,Label,News,Author,Date,URL,Authenticity,non_punc,tamil_tokenized_text,clean
0,Actor Jayam Ravi has Corona,entertainment,நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,0,நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]",நடிகர் ஜெயம் ரவிக்கு
1,A football god about the 'hands of God',miscellaneous,‘கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,ஆதி,01-Dec-20,https://www.hindutamil.in/news/supplements/ila...,0,கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்,"[கடவுளின், கை’களைப், பற்றிய, கால்பந்துக், கடவுள்]",கடவுளின் கை’களைப் பற்றிய கால்பந்துக் கடவுள்
2,Cameras mandatory in school buses: Tamil Nadu ...,politics,பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,செய்திப்பிரிவு,21-Oct-22,https://www.hindutamil.in/news/tamilnadu/88576...,0,பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரச...,"[பள்ளி, பேருந்துகளில், கேமரா, கட்டாயம்:, தமிழக...",பள்ளி பேருந்துகளில் கேமரா கட்டாயம்: தமிழக அரசு...
3,36th National Games | 4 gold medals for Tamil ...,sport,36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,செய்திப்பிரிவு,02-Oct-22,https://www.hindutamil.in/news/sports/877427-3...,0,36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்த...,"[36-வது, தேசிய, விளையாட்டு, |, ஒரே, நாளில், தம...",36-வது தேசிய விளையாட்டு | ஒரே நாளில் தமிழகத்து...
4,Introducing NoiseFit Core 2 Smartwatch: Amazin...,tech,நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,செய்திப்பிரிவு,22-Aug-22,https://www.hindutamil.in/news/technology/8467...,0,நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 5...,"[நாய்ஸ்ஃபிட், கோர், 2, ஸ்மார்ட்வாட்ச், அறிமுகம...",நாய்ஸ்ஃபிட் கோர் 2 ஸ்மார்ட்வாட்ச் அறிமுகம்: 50...


In [38]:
three_shot['clean'] = three_shot['tamil_tokenized_text'].apply(lambda x: ' '.join(x))

In [39]:
three_shot_x, tfidf_vectorizer = tfidf(three_shot['clean'])
three_shot_test = tfidf_vectorizer.transform(xtest)

In [42]:
#Run the machine learning models for one-shot
#Logistic Regression with TF-IDF
clf = LogisticRegression(C=1.0)
clf.fit(three_shot_x, three_shot['Authenticity'])
predictions = clf.predict_proba(three_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.7208413001912046

In [43]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.63      0.71      0.67       207
           1       0.79      0.73      0.76       316

    accuracy                           0.72       523
   macro avg       0.71      0.72      0.71       523
weighted avg       0.73      0.72      0.72       523



In [44]:
#Naive Bayes
clf = MultinomialNB(alpha=1.0)
clf.fit(three_shot_x, three_shot['Authenticity'])
predictions = clf.predict_proba(three_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.7093690248565966

In [45]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.65      0.68      0.67       223
           1       0.76      0.73      0.74       300

    accuracy                           0.71       523
   macro avg       0.70      0.71      0.70       523
weighted avg       0.71      0.71      0.71       523



In [46]:
#Support Vector Machine
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(three_shot_x, three_shot['Authenticity'])
predictions = clf.predict_proba(three_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.6883365200764818

In [47]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.52      0.71      0.60       170
           1       0.83      0.68      0.75       353

    accuracy                           0.69       523
   macro avg       0.67      0.69      0.67       523
weighted avg       0.73      0.69      0.70       523



In [48]:
#XG-Boost classifier
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(three_shot_x, three_shot['Authenticity'])
predictions = clf.predict_proba(three_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy


0.5066921606118547

In [49]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.12      0.35      0.18        81
           1       0.82      0.54      0.65       442

    accuracy                           0.51       523
   macro avg       0.47      0.44      0.41       523
weighted avg       0.71      0.51      0.57       523



### Five-Shot dataset

In [50]:
#Apply TF-IDF to the dataset
tamil_preprocessing(five_shot)
five_shot.head()

,English version,Label,News,Author,Date,URL,Authenticity,non_punc,tamil_tokenized_text
0,Actor Jayam Ravi has Corona,entertainment,நடிகர் ஜெயம் ரவிக்கு,தினத்தந்தி,22-10-2022 02:53,https://www.dailythanthi.com/Cinema/CinemaNews...,0,நடிகர் ஜெயம் ரவிக்கு,"[நடிகர், ஜெயம், ரவிக்கு]"
1,Pooja Hegde regrets the film's failure,entertainment,படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,தினத்தந்தி,22-10-2022 02:50,https://www.dailythanthi.com/Cinema/CinemaNews...,0,படங்கள் தோல்வியால் வருந்தும் பூஜா ஹெக்டே,"[படங்கள், தோல்வியால், வருந்தும், பூஜா, ஹெக்டே]"
2,Images for Diwali,entertainment,தீபாவளிக்கு வரும் படங்கள்,தினத்தந்தி,17-10-2022 10:20,https://www.dailythanthi.com/Cinema/CinemaNews...,0,தீபாவளிக்கு வரும் படங்கள்,"[தீபாவளிக்கு, வரும், படங்கள்]"
3,Simbu debuts in Bollywood,entertainment,பாலிவுட்டில் அறிமுகமாகிறார் சிம்பு,தினத்தந்தி,16-10-2022 03:49,https://www.dailythanthi.com/Cinema/CinemaNews...,0,பாலிவுட்டில் அறிமுகமாகிறார் சிம்பு,"[பாலிவுட்டில், அறிமுகமாகிறார், சிம்பு]"
4,3rd song release from 'Prince',entertainment,'பிரின்ஸ்' படத்தின் 3வது பாடல் வெளியீடு,தினத்தந்தி,14-10-2022 16:31,https://www.dailythanthi.com/Cinema/CinemaNews...,0,பிரின்ஸ் படத்தின் 3வது பாடல் வெளியீடு,"[பிரின்ஸ், படத்தின், 3வது, பாடல், வெளியீடு]"


In [51]:
five_shot['clean'] = five_shot['tamil_tokenized_text'].apply(lambda x: ' '.join(x))

In [52]:
five_shot_x, tfidf_vectorizer = tfidf(five_shot['clean'])
five_shot_test = tfidf_vectorizer.transform(xtest)

In [53]:
#Run the machine learning models for one-shot
#Logistic Regression with TF-IDF
clf = LogisticRegression(C=1.0)
clf.fit(five_shot_x, five_shot['Authenticity'])
predictions = clf.predict_proba(five_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.768642447418738

In [54]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.67      0.78      0.72       200
           1       0.85      0.76      0.80       323

    accuracy                           0.77       523
   macro avg       0.76      0.77      0.76       523
weighted avg       0.78      0.77      0.77       523



In [55]:
#Naive Bayes
clf = MultinomialNB(alpha=1.0)
clf.fit(five_shot_x, five_shot['Authenticity'])
predictions = clf.predict_proba(five_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.7380497131931166

In [56]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.60      0.76      0.67       184
           1       0.85      0.73      0.78       339

    accuracy                           0.74       523
   macro avg       0.72      0.74      0.73       523
weighted avg       0.76      0.74      0.74       523



In [57]:
#Support Vector Machine
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(five_shot_x, five_shot['Authenticity'])
predictions = clf.predict_proba(five_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy

0.762906309751434

In [58]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.66      0.77      0.71       199
           1       0.84      0.76      0.80       324

    accuracy                           0.76       523
   macro avg       0.75      0.77      0.76       523
weighted avg       0.77      0.76      0.77       523



In [59]:
#XG-Boost classifier
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(five_shot_x, five_shot['Authenticity'])
predictions = clf.predict_proba(five_shot_test)
predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

accuracy = sklearn.metrics.accuracy_score(predictions,ytest)
accuracy


0.6328871892925431

In [60]:
print(classification_report(predictions, ytest))

              precision    recall  f1-score   support

           0       0.67      0.58      0.62       273
           1       0.60      0.70      0.64       250

    accuracy                           0.63       523
   macro avg       0.64      0.64      0.63       523
weighted avg       0.64      0.63      0.63       523

